# Identify app types that attract users in iOS & Android stores
Look into app listing in Apple app store and Andoird Play Store to identify free app types that garner the most # of users

In [1]:
open_apple_file = open('AppleStore.csv')
from csv import reader
read_apple_file = reader(open_apple_file)
apps_list = list(read_apple_file)

open_android_file = open('googleplaystore.csv')
from csv import reader
read_android_file = reader(open_android_file)
android_list = list(read_android_file)

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(apps_list, 0, 2, rows_and_columns = True)
explore_data(android_list, 0, 2, rows_and_columns = True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


While in lesson 2, the question was 'Print the column names and try to identify the columns that could help us with our analysis. Use the documentation of the data sets if you're having trouble understanding what a column describes. Add a link to the documentation for readers if you think the column names are not descriptive enough.'

For Apple app store, potentially interesting columns would be - price (Column # 4, python index 3) and prime_genre (Column # 12, Python index 11).

For Android Play store, potentially interesting columns would be - Type (Column # 7, Python index # 6), Price (Column # 8, Python index # 7), Category (Column # 2, Python index # 1) and Genres (Column # 10, Python index # 9).
Type - Free or Paid.
Price - Self-explnatory.
Category and Genres seems to be the same.

Based on kaggle, it seems there is one row with column shift, meaning 'the total number of values' in that particular row is less than the total # of values in the header or rest of the rows. The goal is to identify and remove it.

This was done by using the length of header row for reference. The row ID was identified using a counter and then removed below.

In [2]:
## Code to identify the row with less # of values than the header aka column shift
android_header_length = len(android_list[0])
counter = 0
for temp in android_list:
    row_length = len(temp)
    if row_length != android_header_length:
        print(temp)
        print('Python row index = ', counter)
##        del temp
    counter += 1
    

## print(android_list[10472], android_list[10473], android_list[10474], '\n')

del android_list[10473]

## print(android_list[10472], android_list[10473], android_list[10474])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
Python row index =  10473


In [3]:
duplicate_id = []
unique_id = []

for temp in android_list[1:]:
    if temp[0] in unique_id:
        duplicate_id.append(temp[0])
    else:
        unique_id.append(temp[0])
        
print('Total # of duplicates in Android Play Store =', len(duplicate_id))

##for temp in android_list[1:]:
##    if temp[0] == duplicate_id[0]:
##        print(temp, '\n')

##for temp in duplicate_id:
##    for temp1 in android_list[1:]:
##        if temp == temp1[0]:
 ##           print(temp1)

Total # of duplicates in Android Play Store = 1181


Based on the code in the above code box + the output, Google Play store has 1181 duplicates. The duplicates tend to have the same values except for the column, 'Reviews'. 

1. Our first approach is to use total number of reviews (4th column), identify the maximum value and utilize it. Rest of the duplicate rows with lower # of reviews are not being used.
2. Another hypothese is that we are not interested in the value within the 'Reviews' column. Even if we are, the values seem to not differ much. Hence the plan is to remove all the rows except the first occuring one. This is just a hypothesis and may not even be done.

Create an empty dictionary. The intent is to create a dictionary with unique app names as keys in the dictionary. The corresponding values to each key would be the maximum number of reviews for the app.

Iterate through the list.
    If the name is not in the dictionary, add the app name and the total number of reviews as a float value.
    If the name is already in the dictionary, compare the already existing number of reviews in the dictionary against the repeating app name's number of reviews. If the number of reviews value in the dictionary is lesser, replace that with the higher value existing in the newly found repeated app name's number of reviews.
    
Verify our work by ensuring that total number of value in the dictionary to be 9659 (10842 - 1 - 1 - 1181).
Total number of rows in the first cell - 10842
Subtract header - 1
Subtract the deleted row due to column shift in the second cell - 1
Subtract the total number of duplicates - 1181

Intent of the following code is to identify app names and the respective maximum # of reviews. These are stored in a dictionary.

Loop through Andoid play list. If the name is already in the dictionary, compare the review value tied to the app name in the dictionary against the review in the play list, If the value in play list is higher, update the review value in the dictionary. The values are compared after converting it into float.

In [4]:
reviews_max = {}

for temp in android_list[1:]:
    name = temp[0]
    n_reviews = float(temp[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
    
        
print(len(reviews_max))
##print(reviews_max)

9659


Original intent was to identify the row ID and delete it using del common. However, it was found that the row ID # would reduce and may become cumbersome. Instead the game plan is to copy the unique IDs and the repeated IDs with the maximum review # to a new list. Break statement was used as a list may have app IDs repeat with repeating review #s.

In [5]:
android_clean = []

for temp_1 in reviews_max:
##    print(temp_1)
    for temp_2 in android_list[1:]:
        if temp_1 == temp_2[0] and reviews_max[temp_1] == float(temp_2[3]):
            android_clean.append(temp_2)
            break
            
print(len(android_clean))

9659


Intent is to see if there are any column shifts and duplicates in the Apple app store.

In [6]:
## Code to identify the row with less # of values than the header aka column shift
apple_header_length = len(apps_list[0])
counter = 0
for temp in apps_list:
    row_length = len(temp)
    if row_length != apple_header_length:
        print(temp)
        print('Python row index = ', counter)
##        del temp
    counter += 1
    

## print(android_list[10472], android_list[10473], android_list[10474], '\n')

## del android_list[10473]

## print(android_list[10472], android_list[10473], android_list[10474])

No results from the above code confirm that app store list does not have any column shifts.

In [7]:
duplicate_id = []
unique_id = []

for temp in apps_list[1:]:
    if temp[1] in unique_id:
        duplicate_id.append(temp[0])
    else:
        unique_id.append(temp[0])
        
print('Total # of duplicates in Apple app store =', len(duplicate_id))

Total # of duplicates in Apple app store = 0


Results from the above code confirm that there are no duplicates in the Apple app store.

Hooks like Google Play Store and Apple Apps Store tend to have non-English apps. Our intent is to build apps for English audience. Hence our plan is to write a function which takes in the app name and resturn True if all the characters are English. If any of the character is non-English, it shall return False. There are 2 items to keep in mind. Use ord() function to find the # equivalent for the character. Some English apps may have smilies and other characters. Hence it is ok to have upto 3 non-English characters. This is a decent approximation.

Once a App is identified move this to a seperate list for both the app list.

In [8]:
abc = ['Instagram', '爱奇艺PPS -《欢乐颂2》电视剧热播', 'Docs To Go™ Free Office Suite', 'Instachat 😜', '爱奇艺PPS', '爱奇艺PPS奇']

def non_english(string):
    counter = 0
    for temp in string:
##        print(temp)
        if ord(temp) > 127:
            counter += 1
        if counter > 3:
            return False
    return True

android_clean_english = []

for temp_1 in android_clean:
##    print(temp_1)
    if non_english(temp_1[0]):
        android_clean_english.append(temp_1)
##        print('English')
##    else:
##        print('Non English')

print('Total # of English records within Android Play list =', len(android_clean_english))

apple_clean_english = []

for temp_1 in apps_list[1:]:
    if non_english(temp_1[1]):
        apple_clean_english.append(temp_1)
        
print('Total # of English records within Apple Apps list =', len(apple_clean_english))

Total # of English records within Android Play list = 9614
Total # of English records within Apple Apps list = 6183


Identify free apps and move it to a seperate list.

In [9]:
android_clean_english_free = []

for temp in android_clean_english:
##    print(temp)
    if temp[7] == '0':
        android_clean_english_free.append(temp)
##        print('Free')

print('Total # of free English apps in Android Play Store =', len(android_clean_english_free))

apple_clean_english_free = []

for temp in apple_clean_english:
##    print(temp)
    if temp[4] == '0.0':
        apple_clean_english_free.append(temp)
##        print('Free')

print('Total # of free English apps in Apple Apps Store =', len(apple_clean_english_free))

Total # of free English apps in Android Play Store = 8864
Total # of free English apps in Apple Apps Store = 3222


Our goal is to find app genres that are common with both the app stores.

In [11]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
    print('\n')

def freq_table(list1, index):
    freq_dictionary = {}
    for temp in list1:
        if temp[index] not in freq_dictionary:
            freq_dictionary[temp[index]] = 1
        else:
            freq_dictionary[temp[index]] += 1
    
    sum = 0
    for temp in freq_dictionary:
        sum = sum + freq_dictionary[temp]
##    print(sum)
    
    for temp in freq_dictionary:
        freq_dictionary[temp] = freq_dictionary[temp] / sum * 100

    return freq_dictionary

print('Frequency of Apple app types based on free English apps')
display_table(apple_clean_english_free, 11)

print('Frequency of Android app types based on free English apps. Using Category')
display_table(android_clean_english_free, 1)

print('Frequency of Android app types based on free English apps. Using Genres')
display_table(android_clean_english_free, 9)


## example_test = [['abc', 'A'], ['bbc', 'A'], ['bac', 'A'], ['abs', 'B'], ['sdf', 'A'], ['wer', 'B']]

## android_freq_categ = freq_table(android_clean_english_free, 1)
## print(android_freq_categ)

## android_freq_genres = freq_table(android_clean_english_free, 9)
## print(android_freq_genres)

## apple_freq_prime_genre = freq_table(apple_clean_english_free, 11)
## print(apple_freq_prime_genre)

Frequency of Apple app types based on free English apps
Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Frequency of Android app types based on free English apps. Using Category
FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.89214801

Based on the analysis above, it seems Games to be the most common app type. Just because these are the app types, that does not mean those are the most used. We may use the most commonly downloaded app type and figure this out by looking at the # of downloads. If downloads are not available, # of reviews may be used as proxies.

In [42]:
apple_freq_table = {}
apple_review_total = {}
apple_avg_review = {}

def freq_table1(list1, index):
    freq_dictionary = {}
    for temp in list1:
        if temp[index] not in freq_dictionary:
            freq_dictionary[temp[index]] = 1
        else:
            freq_dictionary[temp[index]] += 1
    return freq_dictionary

apple_freq_table = freq_table1(apple_clean_english_free, 11)
print(apple_freq_table)

for temp in apple_clean_english_free:
    if temp[11] not in apple_review_total:
        apple_review_total[temp[11]] = int(temp[5])
    else:
        apple_review_total[temp[11]] += int(temp[5])
        
print(apple_review_total)

for temp in apple_freq_table:
    #print(temp)
    #print(apple_review_total[temp])
    #print(apple_avg_review[temp])
    apple_avg_review[temp] = apple_review_total[temp] / apple_freq_table[temp]
    
print(apple_avg_review)

{'Social Networking': 106, 'Photo & Video': 160, 'Games': 1874, 'Music': 66, 'Reference': 18, 'Health & Fitness': 65, 'Weather': 28, 'Utilities': 81, 'Travel': 40, 'Shopping': 84, 'News': 43, 'Navigation': 6, 'Lifestyle': 51, 'Entertainment': 254, 'Food & Drink': 26, 'Sports': 69, 'Book': 14, 'Finance': 36, 'Education': 118, 'Productivity': 56, 'Business': 17, 'Catalogs': 4, 'Medical': 6}
{'Social Networking': 7584125, 'Photo & Video': 4550647, 'Games': 42705967, 'Music': 3783551, 'Reference': 1348958, 'Health & Fitness': 1514371, 'Weather': 1463837, 'Utilities': 1513441, 'Travel': 1129752, 'Shopping': 2261254, 'News': 913665, 'Navigation': 516542, 'Lifestyle': 840774, 'Entertainment': 3563577, 'Food & Drink': 866682, 'Sports': 1587614, 'Book': 556619, 'Finance': 1132846, 'Education': 826470, 'Productivity': 1177591, 'Business': 127349, 'Catalogs': 16016, 'Medical': 3672}
{'Social Networking': 71548.34905660378, 'Photo & Video': 28441.54375, 'Games': 22788.6696905016, 'Music': 57326.53